### Setup and Configuration

In [ ]:
!pip install spotipy
!pip install google-api-python-client
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [ ]:
# Import libraries
import requests
from google import genai
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import random
from googleapiclient.discovery import build
import os

In [ ]:
# API Configuration

GOOGLE_BOOKS_API_KEY = "GOOGLE_BOOKS_API_KEY"
GEMINI_API_KEY = "GEMINI_API_KEY"
SPOTIFY_CLIENT_ID = "SPOTIFY_CLIENT_ID"
SPOTIFY_CLIENT_SECRET = "SPOTIFY_CLIENT_SECRET"
REDIRECT_URI = "REDIRECT_URI"

### Fetch Book Summary

In [ ]:
def get_book_info(title, author):
    """Fetch book information from Google Books API"""
    query = f"intitle:{title}+inauthor:{author}"
    url = f"https://www.googleapis.com/books/v1/volumes?q={query}&key={GOOGLE_BOOKS_API_KEY}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if "items" not in data or not data["items"]:
            raise ValueError("No books found for the provided title and author.")

        book = data["items"][0]["volumeInfo"]
        return {
            "title": book.get("title", "Unknown Title"),
            "authors": book.get("authors", ["Unknown Author"]),
            "summary": book.get("description", "No description available."),
        }

    except Exception as e:
        print(f"Error fetching book info: {e}")
        return None

### Analyze Book Content

In [ ]:
def analyze_book_with_gemini(book_info):
    """Analyze book content using Google's Gemini AI"""
    client = genai.Client(api_key=GEMINI_API_KEY)

    prompt = f"""
    You are a music and literature expert. Based on the book details below, extract musical insights that would help create a fitting and emotionally resonant soundtrack.

    Title: {book_info['title']}
    Author(s): {', '.join(book_info['authors'])}
    Summary: {book_info['summary']}

    Please analyze the story and return:

    1. **Emotional Tones** – The dominant emotional qualities of the book
    2. **Genres** – Suitable music genres that match the overall tone and pacing
    3. **Moods** – Key moods or emotional shifts across the book
    4. **Time Period / Cultural Context** – Historical or cultural elements
    5. **Keywords** – 5–7 vivid, descriptive words ideal for searching music

    Respond strictly in this format:
    Emotional Tones: [comma-separated list]
    Genres: [comma-separated list]
    Moods: [comma-separated list]
    Time Period/Cultural Context: [brief description]
    Keywords: [comma-separated list]
    """

    try:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt
        )
        return response.text
    except Exception as e:
        print(f"Error with Gemini API: {e}")
        return None

In [ ]:
def parse_gemini_analysis(analysis_text):
    """Parse the structured response from Gemini into a dictionary"""
    if not analysis_text:
        return None

    try:
        sections = {}
        for line in analysis_text.split('\n'):
            line = line.strip()
            if not line or ':' not in line:
                continue

            section_name, content = line.split(':', 1)
            section_name = section_name.strip()
            content = content.strip()

            if content.startswith('[') and content.endswith(']'):
                content = content[1:-1]
            sections[section_name] = [item.strip() for item in content.split(',')]

        return sections
    except Exception as e:
        print(f"Error parsing Gemini analysis: {e}")
        return None

### Generate Spotify Playlist

In [ ]:
def setup_spotify_client():
    """Set up authenticated Spotify client"""
    try:
        auth_manager = SpotifyOAuth(
            client_id=SPOTIFY_CLIENT_ID,
            client_secret=SPOTIFY_CLIENT_SECRET,
            redirect_uri=REDIRECT_URI,
            scope="playlist-modify-public",
            open_browser=False
        )

        auth_url = auth_manager.get_authorize_url()
        print(f"\nPlease visit this URL to authorize the application:\n{auth_url}")
        print("\nAfter authorizing, paste the redirect URL here:")
        response_url = input().strip()

        code = auth_manager.parse_response_code(response_url)
        token_info = auth_manager.get_access_token(code)

        return spotipy.Spotify(auth=token_info['access_token'])

    except Exception as e:
        print(f"Error in Spotify authentication: {e}")
        return None

In [ ]:
def search_spotify_tracks(sp, analysis, max_tracks=15):
    """Search for tracks based on book analysis"""
    if not sp or not analysis:
        return []

    tracks = []
    search_weights = {
        'Emotional Tones': 0.3,
        'Moods': 0.3,
        'Genres': 0.25,
        'Keywords': 0.15
    }

    # Build weighted search queries
    search_queries = []

    # Genre queries
    if 'Genres' in analysis:
        for genre in analysis['Genres']:
            search_queries.append({
                'terms': f'genre:"{genre}"',
                'limit': int(max_tracks * search_weights['Genres'])
            })

    # Emotion-genre combinations
    if 'Emotional Tones' in analysis and 'Genres' in analysis:
        for tone in analysis['Emotional Tones']:
            for genre in analysis['Genres'][:2]:
                search_queries.append({
                    'terms': f'{tone} {genre}',
                    'limit': int(max_tracks * search_weights['Emotional Tones'])
                })

    # Mood queries
    if 'Moods' in analysis:
        for mood in analysis['Moods']:
            search_queries.append({
                'terms': mood,
                'limit': int(max_tracks * search_weights['Moods'])
            })

    # Keyword queries
    if 'Keywords' in analysis:
        for keyword in analysis['Keywords']:
            search_queries.append({
                'terms': keyword,
                'limit': int(max_tracks * search_weights['Keywords'])
            })

    # Execute searches
    for query in search_queries:
        try:
            results = sp.search(
                q=query['terms'],
                type="track",
                limit=query['limit']
            )

            for item in results["tracks"]["items"]:
                track = {
                    "id": item["id"],
                    "name": item["name"],
                    "artist": item["artists"][0]["name"],
                    "album": item["album"]["name"],
                    "uri": item["uri"],
                    "popularity": item["popularity"]
                }
                if track not in tracks:
                    tracks.append(track)

        except Exception as e:
            print(f"Error searching for '{query['terms']}': {e}")
            continue

    # Sort by popularity and return top tracks
    tracks.sort(key=lambda x: x['popularity'], reverse=True)
    return tracks[:max_tracks]

In [ ]:
def create_spotify_playlist(sp, book_info, tracks, analysis):
    """Create a Spotify playlist from selected tracks"""
    if not sp or not tracks:
        return None

    try:
        user_id = sp.current_user()["id"]

        # Create playlist with rich metadata
        playlist_name = f"📚 {book_info['title']} - Literary Soundtrack"

        description_elements = []
        if analysis.get('Genres'):
            description_elements.append(f"Genres: {', '.join(analysis['Genres'][:3])}")
        if analysis.get('Moods'):
            description_elements.append(f"Moods: {', '.join(analysis['Moods'][:3])}")

        playlist_description = (
            f"A curated soundtrack for {book_info['title']} "
            f"by {', '.join(book_info['authors'])}. "
            f"{' | '.join(description_elements)}"
        )

        # Create playlist
        playlist = sp.user_playlist_create(
            user_id,
            playlist_name,
            public=True,
            description=playlist_description[:300]
        )

        # Add tracks in batches
        track_uris = [track["uri"] for track in tracks]
        batch_size = 50
        for i in range(0, len(track_uris), batch_size):
            batch = track_uris[i:i + batch_size]
            sp.playlist_add_items(playlist["id"], batch)

        return playlist["external_urls"]["spotify"]

    except Exception as e:
        print(f"Error creating playlist: {e}")
        return None

### Run Full Pipeline

In [ ]:
def display_analysis_results(analysis):
    """Display the parsed analysis results in a structured format."""
    print("\nAnalysis Results:")
    print("=" * 50)

    sections = {
        'Emotional Tones': '🎭 Emotional Tones',
        'Genres': '🎵 Musical Genres',
        'Moods': '🌈 Moods',
        'Time Period/Cultural Context': '📅 Time Period/Cultural Context',
        'Keywords': '🔑 Keywords'
    }

    for section, header in sections.items():
        if section in analysis:
            print(f"\n{header}:")
            if isinstance(analysis[section], list):
                print("  • " + "\n  • ".join(analysis[section]))
            else:
                print("  • " + str(analysis[section]))
    print("\n" + "=" * 50)

def display_tracks_details(tracks):
    """Display track information in a formatted way."""
    print("\nSelected Tracks:")
    print("=" * 50)

    for i, track in enumerate(tracks, 1):
        popularity_stars = "⭐" * ((track['popularity'] // 20) + 1)
        print(f"\n{i}. {track['name']}")
        print(f"   Artist: {track['artist']}")
        print(f"   Album: {track['album']}")
        print(f"   Popularity: {popularity_stars} ({track['popularity']}/100)")

    print("\n" + "=" * 50)

In [ ]:
def main():
    """Enhanced main function to run the book playlist generator."""
    print("📚 Welcome to the Literary Soundtrack Generator! 🎵")
    print("=" * 50)

    # Get book details with input validation
    while True:
        book_title = input("\nEnter the book title: ").strip()
        if book_title:
            break
        print("Please enter a valid book title.")

    while True:
        book_author = input("Enter the book's author: ").strip()
        if book_author:
            break
        print("Please enter a valid author name.")

    # Fetch book information
    print("\n📖 Fetching book information...")
    book_info = get_book_info(book_title, book_author)

    if not book_info:
        print("❌ Could not find book information. Please check the title and author.")
        return

    print(f"\n✅ Book found: {book_info['title']} by {', '.join(book_info['authors'])}")
    if book_info.get('summary'):
        print("\nSummary:")
        print("-" * 50)
        print(f"{book_info['summary'][:200]}...")
        print("-" * 50)

    # Analyze book content
    print("\n🤖 Analyzing book content using AI...")
    analysis_text = analyze_book_with_gemini(book_info)

    if not analysis_text:
        print("❌ Failed to analyze the book. Please try again.")
        return

    # Parse the analysis
    analysis = parse_gemini_analysis(analysis_text)
    if not analysis:
        print("❌ Failed to parse the analysis results. Please try again.")
        return

    # Display the analysis results
    display_analysis_results(analysis)

    # Set up Spotify
    print("\n🎵 Setting up Spotify connection...")
    sp = setup_spotify_client()

    if not sp:
        print("❌ Failed to set up Spotify client. Please check your credentials and try again.")
        return

    # Search for tracks
    print("\n🔍 Searching for matching tracks...")
    tracks = search_spotify_tracks(sp, analysis)

    if not tracks:
        print("❌ No tracks found. Please try again with different search terms.")
        return

    # Display found tracks with details
    display_tracks_details(tracks)

    # Confirm playlist creation
    while True:
        create = input("\nWould you like to create a Spotify playlist with these tracks? (y/n): ").lower()
        if create in ['y', 'n']:
            break
        print("Please enter 'y' for yes or 'n' for no.")

    if create == 'n':
        print("\n👋 Thanks for using the Literary Soundtrack Generator!")
        return

    # Create playlist
    print("\n📝 Creating Spotify playlist...")
    playlist_url = create_spotify_playlist(sp, book_info, tracks, analysis)

    if playlist_url:
        print(f"\n🎉 Success! Your literary soundtrack is ready!")
        print(f"🔗 Playlist URL: {playlist_url}")
        print("\nTip: The playlist is public - you can share it with others!")
    else:
        print("\n❌ Failed to create playlist. Please try again.")

    print("\n👋 Thanks for using the Literary Soundtrack Generator!")

In [ ]:
if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\n👋 Program interrupted. Thanks for using the Literary Soundtrack Generator!")
    except Exception as e:
        print(f"\n❌ An unexpected error occurred: {str(e)}")
        print("Please try again or contact support if the issue persists.")

📚 Welcome to the Literary Soundtrack Generator! 🎵

Enter the book title: Perahu Kertas
Enter the book's author: Dee Lestari

📖 Fetching book information...

✅ Book found: Perahu Kertas by Dee Lestari

Summary:
--------------------------------------------------
Namanya Kugy. Mungil, pengkhayal, dan berantakan. Dari benaknya, mengalir untaian dongeng indah. Keenan belum pernah bertemu manusia seaneh itu .... Namanya Keenan. Cerdas, artistik, dan penuh kejutan...
--------------------------------------------------

🤖 Analyzing book content using AI...

Analysis Results:

🎭 Emotional Tones:
  • Romantic
  • whimsical
  • hopeful
  • dreamy
  • melancholic
  • bittersweet

🎵 Musical Genres:
  • Indonesian Pop
  • Acoustic Pop
  • Indie Pop
  • Ballads
  • Light Orchestral
  • Folk

🌈 Moods:
  • Innocent wonder
  • budding romance
  • creative inspiration
  • facing obstacles
  • yearning
  • resolution

📅 Time Period/Cultural Context:
  • Contemporary Indonesia
  • reflecting modern Indonesi

<ipython-input-10-3322e05810bb>:18: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = auth_manager.get_access_token(code)



🔍 Searching for matching tracks...

Selected Tracks:

1. Somewhere Only We Know
   Artist: Keane
   Album: Hopes And Fears
   Popularity: ⭐⭐⭐⭐⭐ (88/100)

2. Lesung Pipi
   Artist: Raim Laode
   Album: Lesung Pipi
   Popularity: ⭐⭐⭐⭐⭐ (86/100)

3. Komang
   Artist: Raim Laode
   Album: Komang
   Popularity: ⭐⭐⭐⭐⭐ (85/100)

4. Mangu
   Artist: Fourtwnty
   Album: Mangu
   Popularity: ⭐⭐⭐⭐⭐ (84/100)

5. Lover, You Should've Come Over
   Artist: Jeff Buckley
   Album: Grace
   Popularity: ⭐⭐⭐⭐⭐ (83/100)

6. Mangu
   Artist: Fourtwnty
   Album: Nalar
   Popularity: ⭐⭐⭐⭐⭐ (82/100)

7. Hold On
   Artist: Chord Overstreet
   Album: Hold On
   Popularity: ⭐⭐⭐⭐⭐ (81/100)

8. Nanti Kita Seperti Ini
   Artist: Batas Senja
   Album: Nanti Kita Seperti Ini
   Popularity: ⭐⭐⭐⭐⭐ (81/100)

9. La La Lost You - Acoustic Version
   Artist: NIKI
   Album: NIKI Acoustic Sessions: Head In The Clouds II
   Popularity: ⭐⭐⭐⭐⭐ (80/100)

10. Angel Numbers / Ten Toes
   Artist: Chris Brown
   Album: 11:11
   Popu